In [22]:
import string
import pandas as pd
import numpy as np

from collections import Counter
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import linear_model
from sklearn.model_selection import train_test_split

np.set_printoptions(precision=5)
%matplotlib inline
pd.options.display.max_columns=1000

Source: http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/

In [23]:
df = pd.read_csv('SMSSpamCollection.txt', sep='\t', header=None)

In [24]:
df.columns = ['spam', 'text']

In [25]:
df.head()

,spam,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [26]:
# Get rid of the punctuation
translator = str.maketrans('', '', string.punctuation)
df.text = df.text.apply(lambda x: x.translate(translator))
df.head()

,spam,text
0,ham,Go until jurong point crazy Available only in ...
1,ham,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor U c already then say
4,ham,Nah I dont think he goes to usf he lives aroun...


In [27]:
df['spam'] = df['spam'] == 'spam' # makes True/False nstead of "spam" and "ham"
df['spam'] = df['spam'].astype(int)  # number values instead of boolean value

In [28]:
df.text[0]

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [29]:
df.head()

,spam,text
0,0,Go until jurong point crazy Available only in ...
1,0,Ok lar Joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor U c already then say
4,0,Nah I dont think he goes to usf he lives aroun...


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
spam    5572 non-null int32
text    5572 non-null object
dtypes: int32(1), object(1)
memory usage: 65.4+ KB


In [31]:
# Use TweetTokenizer 
tknzr = TweetTokenizer()
df['text'] = df.text.apply(tknzr.tokenize)
df['text'].head()

0    [Go, until, jurong, point, crazy, Available, o...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, in, 2, a, wkly, comp, to, win, F...
3    [U, dun, say, so, early, hor, U, c, already, t...
4    [Nah, I, dont, think, he, goes, to, usf, he, l...
Name: text, dtype: object

In [32]:
# Adding a new feature 'num_words' - how many words are in a message
df['num_words'] = df.text.apply(len)
df['num_words'].head()

0    20
1     6
2    30
3    11
4    13
Name: num_words, dtype: int64

In [33]:
df.text[0]

['Go',
 'until',
 'jurong',
 'point',
 'crazy',
 'Available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'Cine',
 'there',
 'got',
 'amore',
 'wat']

In [34]:
len(df.text)

5572

In [35]:
len(df.text[5571])

6

In [55]:
df.text[0]

['Go',
 'until',
 'jurong',
 'point',
 'crazy',
 'Available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'Cine',
 'there',
 'got',
 'amore',
 'wat']

In [ ]:
# Here for each element of the list we need to get the number of characters and then sum it up
# Is there to do it more effectively then brute force?

#HELL YEAH! LOOK below. 
#L = []
#for i in range(len(df.text)):
#    sum = 0
#    for j in range(len(df.text[i])):
#        sum += len(df.text[i][j])
#    L.append(sum)  

Here I want to have 'raw' messages after getting rid of punctuation and applying TweetTokenizer. I want it because I'll later use it as to transform_fit tfidf.

In [66]:
documents = []
for i in range(len(df.text)):
    documents.append(join_string(df.text[i]))
        
documents[:5]

['Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat',
 'Ok lar Joking wif u oni',
 'Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive entry questionstd txt rateTCs apply 08452810075 over 18s',
 'U dun say so early hor U c already then say',
 'Nah I dont think he goes to usf he lives around here though']

In [69]:
L = []
for i in documents:
    L.append(len(i))
L[:5]

[102, 23, 151, 43, 59]

In [70]:
# adding a new feature 'length' - how many characters a message have (spaces are not included)
df['length'] = L
df['length'].head()

0    102
1     23
2    151
3     43
4     59
Name: length, dtype: int64

In [71]:
df.columns

Index(['spam', 'text', 'num_words', 'length'], dtype='object')

In [72]:
df.text.head()

0    [Go, until, jurong, point, crazy, Available, o...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, in, 2, a, wkly, comp, to, win, F...
3    [U, dun, say, so, early, hor, U, c, already, t...
4    [Nah, I, dont, think, he, goes, to, usf, he, l...
Name: text, dtype: object

In [73]:
# Creating a new DataFrame df_words from bag of words
df_words = df.text.apply(Counter).apply(dict).values #list of dictoniaries to DF
df_words = pd.DataFrame(list(df_words))

In [74]:
df_words.columns

Index(['0', '0089my', '00Please', '01', '0121 2025050', '01223585236',
       '01223585334', '01256987', '02', '020603',
       ...
       'ü', 'üll', '–', '—', '‘', '’', '“', '…', '┾', '〨ud'],
      dtype='object', length=11662)

In [75]:
df_words['length'] = df['length']
df_words['num_words'] = df['num_words']
df_words['spam'] = df.spam

In [76]:
df_words.columns

Index(['0', '0089my', '00Please', '01', '0121 2025050', '01223585236',
       '01223585334', '01256987', '02', '020603',
       ...
       '–', '—', '‘', '’', '“', '…', '┾', '〨ud', 'num_words', 'spam'],
      dtype='object', length=11664)

In [77]:
df_words = df_words.fillna(0).astype(int)
df_words.head()

,0,0089my,00Please,01,0121 2025050,01223585236,01223585334,01256987,02,020603,0207 153 9153,0207 153 9996,02070836089,02072069400,02073162414,02085076972,020903,03530150,040902,050703,06,060505,061104,07008009200,07046744435,07090201529,07090298926,07099833605,071104,07123456789,07732584351,07734396839,07742676969,07753741225,0776xxxxxxx,07781482378,07786200117,077xxx,078,07801543489,07808,07808247860,07808726822,07815296484,07821230901,0784987,07880867867,0789xxxxxxx,07946746291,0796XXXXXX,07973788240,07XXXXXXXXX,07xxxxxxxxx,08,0800,0800 0721072,0800 169 6031,0800 195 6669,0800 1956669,0800 5050,0800 542 0578,0800 542 0825,08000407165,08000776320,08000839402,08000930705,08000938767,08001950382,08002888812,08002986030,08002986906,08002988890,08006344447,0808 145 4742,08081263000,08081560665,0819,0844,08448350055,08448714184,0845 021 3680,0845 2814032,08450542832,08452810071,08452810073,08452810075,0870,08700469649,08700621170,08701213186,08701237397,08701417012,08701624,08701752560,08701872873,08702411827,08702490080,08702840625,08704439680,08706091795,08707379102,08707500020,08707509020,08707533310,08707808226,08708034412,08708800282,08709222922,08709501522,0870k,08710471114,08712101358,08712103738,08712120250,08712300220,08712317606,08712400200,08712400602,08712400603,08712402050,08712402578,08712402779,08712402902,08712402972,08712404000,08712405020,08712405022,08712460324,08712466669,08712778107,08712778108,08712778109,08714342399,08714712377,08714712379,08714712388,08714712394,08714712412,08714714011,08714719523,08714740323,08714742804,08715203028,08715203649,08715203652,08715203656,08715203677,08715203685,08715203694,08715205273,08715500022,08715705022,08717111821,08717168528,08717205546,08717507382,08717507711,08717509990,08717890890,08717895698,08717898035,08718711108,08718720201,08718723815,08718725756,08718726270,08718726970,08718726971,08718726978,08718727200,08718727868,08718727870,08718728876,08718729755,08718729758,08718730555,08718730666,08718738001,08718738002,08718738034,08719180219,08719180248,08719181259,08719181503,08719181513,08719839835,08719899217,08719899229,08719899230,09,09041940223,09050000301,09050000327,09050000332,09050000460,09050000555,09050000878,09050000928,09050001295,09050001808,09050002311,09050003091,09050005321,09050090044,09050280520,09053750005,09056242159,09057039994,09058091854,09058091870,09058094454,09058094455,09058094507,09058094565,09058094583,09058094594,09058094597,09058094599,09058095107,09058095201,09058097189,09058097218,09058098002,09058099801,09061104276,09061104283,09061209465,09061213237,09061221061,09061221066,09061701444,09061701461,09061701851,09061701939,09061702893,09061743386,09061743806,09061743810,09061743811,09061744553,09061749602,09061790121,09061790125,09061790126,09063440451,09063442151,09063458130,09063463,09064011000,09064012103,09064012160,09064015307,09064017295,09064017305,09064018838,09064019014,09064019788,09065069120,09065069154,09065171142,09065174042,09065394514,09065394973,09065989180,09065989182,09066350750,09066358152,09066358361,09066361921,09066362206,09066362220,09066362231,09066364311,09066364349,09066364589,09066368327,09066368470,09066368753,09066380611,09066382422,09066612661,09066649731,09066660100,09071512432,09071512433,09071517866,09077818151,09090204448,09090900040,09094100151,09094646631,09094646899,09095350301,09096102316,09099725823,09099726395,09099726429,09099726481,09099726553,09111030116,09111032124,09701213186,0ANETWORKS,0Hi,0pwk,0quit,1,10,100,1000,10000,100000,1000CALL,1000s,100603,100pSMS,100percentrealcom,100s,100txtmth,1010,1013,101mega,1030,10803,10K,10am,10am7pm,10am9pm,10k,10mins,10p,10pmin,10ppm,10th,11,1120,113,1131,11414,11414TCRW1,1146,1148,116,1172,118pmsg,11mths,12,120,12000pes,1205,121,1225,123,1230,125,1250,125gift,128,1282EssexCM61XN,12Mths,12hours,12hrs,12mths,12n146tf15,12n146tf150p,12price,13,130,131004,1323,1327,13404,139,140,1405 1680,140ppm,1450,146tf150p,14thMarch,150,150 0087040,1500,150Mtmsgrcvd18,15

In [78]:
df_words.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Columns: 11664 entries, 0 to spam
dtypes: int32(11664)
memory usage: 247.9 MB


In [47]:
df.describe()

,spam,num_words,length
count,5572.000000,5572.000000,5572.000000
mean,0.134063,15.543431,61.609476
std,0.340751,11.287774,45.970978
min,0.000000,0.000000,0.000000
25%,0.000000,7.000000,27.000000
50%,0.000000,12.000000,46.000000
75%,0.000000,23.000000,93.000000
max,1.000000,171.000000,718.000000


In [49]:
# Finally, save our preprocessed data
#df_words.to_csv('SMS_preprocessed.csv')

In [79]:
tfidf = TfidfTransformer()
vectorizer = CountVectorizer(ngram_range=(1, 2))
#  TfidfTransformer takes the raw term frequencies from CountVectorizer 
# as input and transforms them into tf-idfs

In [80]:
# Linear Regression model
model = linear_model.LinearRegression()

In [81]:
X = tfidf.fit_transform(vectorizer.fit_transform(documents)).toarray()
y = np.array(df_words.spam)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [83]:
model.fit(X_train, y_train)
print("Model core for training set:", model.score(X_train, y_train))
print("Model core for testing set:", model.score(X_test, y_test))

Model core for training set: 1.0
Model core for testing set: 0.8264001954024383
